In [ ]:
#load modules
import os
import ee
from scripts import gdrive
import subprocess
import glob
import ipywidgets as widgets
from ipywidgets import HBox
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#initialize earth engine
ee.Initialize()

In [ ]:
#define the constants
START_SEPAL = "the process has been launch on your SEPAL account"
NO_INPUT = "No input have been selected, provide them through step 1."
NO_TASK = "The GEE process has not been completed, launch it or run a status check through step 2."
ALREADY_DONE = "This computation has already been performed\nYou can find your results in the glad_result folder of your computer"
BUTTON_COLOR = 'lightgreen'

In [ ]:
#function 
def merge(filename, alert_map, glad_dir):
    """ merge into a single tif files"""
    
    #create command
    command = [
        'gdal_merge.py',
        '-o', alert_map,
        '-v', '-co', '"COMPRESS=LZW"'
    ]
    
    #add the input files
    for file in glob.glob(glad_dir + filename):
        command.append(file)

    process = subprocess.run(
        command,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        universal_newlines=True
    ) 
    
    print(process.stdout)
    
def clump(alert_map, clump_map):
    """ clump the results"""
    process = subprocess.run(
        [
            'oft-clump', 
            '-i', alert_map, 
            '-o', clump_map
        ],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        universal_newlines=True
    )
    
    print (process.stdout)
    
def calc(clump_map, alert_map, alert_stats):
    """Compute the statistics per each individual clump"""
    process = subprocess.run(
        [
            'oft-his', 
            '-um', clump_map, 
            '-i', alert_map, 
            '-o', alert_stats,
            '-maxval','3'
        
        ],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        universal_newlines=True
    )
    
    print(process.stdout)  
    
def create_result_folder():
    """Create a folder to download the glad images"""
    glad_dir = os.path.join(os.path.expanduser('~'), 'glad_results')+'/'
    if not os.path.exists(glad_dir):
        os.makedirs(glad_dir)
    
    return glad_dir

def download_task_tif(country_code, year):
    drive_handler = gdrive.gdrive()
    files = drive_handler.get_files(construct_filename(country_code, year)) #maybe I shoud use a global variable
    drive_handler.download_files(files, glad_dir)
    
def delete_merged_file(filename, glad_dir):
    #list the input files
    file_list = []
    for file in glob.glob(glad_dir + filename):
        file_list.append(file)
        
    for file in file_list:
        os.remove(file)
        
def construct_filename(country_code, year):
    """return the filename associated with the current task"""
    
    filename = 'alerts_' + country_code + '_' + year
    
    return filename

In [ ]:
#template
output = widgets.Output()

button_postprocess = widgets.Button(description="Start SEPAL process!")
button_postprocess.style.button_color = BUTTON_COLOR

In [ ]:
def check_for_inputs():
    """check if the inputs exist and have a value"""
    
    #I just need to check for existance as the first cell has a default value
    return ('country_code' in globals() and 'year' in globals())

def check_task_status(country_code, year):
    #search for the task in task_list
    filename = construct_filename(country_code, year)
    tasks_list = ee.batch.Task.list()
    current_task = ''
    for task in tasks_list:
        if task.config['description'] == filename:
            current_task = task
            break
            
    #task not launched 
    if current_task == '':
        return False
    
    if current_task.state == "RUNNING":
        return False
    
    return True

def check_for_stats(filename):
    return glob.glob(filename)

def display_graph(filename, country_code, year):
    
    data = np.loadtxt(filename, delimiter=' ')
    
    bins = []
    for i in range(1000):
        bins.append(i*10)

    #need to confirm who's who
    Y_5 = data[:,5]
    Y_5 = np.ma.masked_equal(Y_5,0).compressed()
    maxY5 = np.amax(Y_5)

    Y_4 = data[:,4]
    Y_4 = np.ma.masked_equal(Y_4,0).compressed()
    maxY4 = np.amax(Y_4) 
    
    #plot in kilometers ?

    plt.hist(
        [Y_5, Y_4], 
        label=['confirmed alert', 'likeky alert'], 
        weights=[Y_5, Y_4],color=["#C26449", "#59C266"], 
        bins=bins, 
        density=True, 
        histtype='bar', 
        stacked=True
    )
    plt.xlim(left= 0, right=max(maxY5, maxY4))
    plt.yscale('log')
    plt.legend(loc='upper right')
    plt.title('Distribution of GLAD alerts for '+ country_code + " in " + year)
    plt.xlabel('patch size (px)')
    plt.ylabel('number of pixels (log)')
    plt.show()    

In [ ]:
#function for display
#define the function used to display the process
def display_inputs(country_code, year):
    print ("You have selected the data from {0} for the year {1}.".format(country_code, year))
        
def on_button_clicked(b):
    
    global country_code
    global year
    
    with output:
        #check that there are inputs
        if not check_for_inputs():
            print(NO_INPUT)
            return 0
        
        #define the files variables
        #year and country_code are defined by step 1 cell
        alert_map   = glad_dir + "glad_" +year + "_" + country_code + ".tif"
        clump_map   = glad_dir + "tmp_clump_" + year + "_" + country_code + ".tif"
        alert_stats = glad_dir + "stats_glad_" + year + "_" + country_code + ".txt"
        
        #check that the Gee process is finished 
        if not check_task_status(country_code, year):
            print(NO_TASK)
            return 0
        
        #check that the process is not already done 
        button_postprocess.disabled = True
        print(START_SEPAL)
        if check_for_stats(alert_stats):
            print(ALREADY_DONE)
        else:        
            #start the postprocessing
    
            #download from GEE
            download_task_tif(country_code, year)
        
            #process data with otf
            filename = construct_filename(country_code, year) + "*.tif"
            merge(filename, alert_map, glad_dir)
            delete_merged_file(filename, glad_dir)
            clump(alert_map, clump_map)
            calc(clump_map, alert_map, alert_stats)
        
        display_graph(alert_stats, country_code, year)
        button_postprocess.disabled = False
        
        return 1

In [ ]:
#run
glad_dir = create_result_folder()

button_postprocess.on_click(on_button_clicked)
ui_posprocess = HBox([button_postprocess])
display(ui_posprocess)
display(output)